# 5.5 LeNet

卷积层：

+ 卷积层保留输入形状，使图像的像素在高和宽两个方向上的相关性被保留，从而被有效识别。

+ 卷积层使用滑动窗口将同一卷积核不同位置的输入重复计算，避免了参数的尺寸过大。

卷积层用来识别图像中的空间模式，比如线条和物体的局部。

最大池化层用来降低卷积层对于位置的敏感性。

LeNet网络

+ LeNet网络使用5\*5的卷积核，在输出上使用sigmoid激活函数，

+ 第一个卷积层的输出通道为6，第二个卷积层的输出通道为16，因为第二个卷积层输入的高和宽要比第一个的输入小。
> 增加输出通道是为了使两个卷积层的参数尺寸类似。

+ 卷积层输出通道数取决于卷积核的数量，

+ 一个卷积层和一个池化层共同构成卷积块，

+ 卷积块中的最大池化层的窗口为2\*2，步幅为2
> 池化窗口的大小和步幅大小相同，所以池化层在输入上每次滑动所覆盖的区域互不重叠。

+ 一共10个类别，即最后的输出向量维度为10.



其它

+ LeNet中的卷积层，使用的是conv2d()，二维卷积用于图像数据，对宽度和高度进行卷积。`class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)`

+ conv1d()该函数是对文本数据进行卷积，不对高度进行卷积。`class torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)`


In [3]:
import time
import torch
from torch import nn,optim
import d2l_pytorch as d2l 

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        # 卷积块包含2层：卷积层，激活函数，最大池化，卷积层，激活函数，最大池化
        # in_channel=1,out_channle=6,kernelz-size=5
        self.conv=nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5),
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5),
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.fc=nn.Sequential(
            nn.Linear(in_features=16*4*4,out_features=120),
            nn.Sigmoid(),
            nn.Linear(in_features=120,out_features=84),
            nn.Sigmoid(),
            nn.Linear(in_features=84,out_features=10)
        )
    def forward(self,img):
        feature=self.conv(img)
        output=self.fc(feature.view(img.shape[0],-1))
        return output

In [7]:
net=LeNet()
print(net)

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [12]:
# 获取数据和训练数据

batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)


In [13]:
# 评估函数，使支持gpu计算。

def evaluate_accuracy(data_iter,net,device=None):
    # gpu
    if device is None and isinstance(net,torch.nn.Module):
        # 如果没有指定device，则使用net的device
        device=list(net.parameters())[0].device
    # 准确率，总数
    acc_sum,n=0.0,0
    # with torch.no_grad： disables tracking of gradients in autograd. 
    # model.eval()： changes the forward() behaviour of the module it is called upon.
    with torch.no_grad():
        for X,y in data_iter:
            if isinstance(net,torch.nn.Module):
                # 评估模式，该模式会关闭dropout
                net.eval()
                # torch.argmax(input, dim, keepdim=False) → LongTensor返回指定维度的最大值的索引。
                acc_sum+=( net(X.to(device)).argmax(dim=1) == y.to(device) ).float().sum().cpu().item()
            else: # 无GPU
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n+=y.shape[0]
    return acc_sum/n

In [18]:
# 训练函数：确保计算使用的数据和模型在同一个内存或显卡上

def train(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs):
    net=net.to(device)
    print('training on ',device)
    # 损失函数，使用交叉熵损失函数
    loss=torch.nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        train_l_sum,train_acc_sum,n,batch_count,start=0.0,0.0,0,0,time.time()
        for X,y in train_iter:
            X=X.to(device)
            y=y.to(device)
            y_hat=net(X)
            l=loss(y_hat,y)
            # 梯度清零
            optimizer.zero_grad()
            # 反向传播
            l.backward()
            # 更新参数
            optimizer.step()
            
            # 更新损失和正确率
            train_l_sum+=l.cpu().item()
            train_acc_sum+=(y_hat.argmax(dim=1) == y ).sum().cpu().item()
            n+=y.shape[0]
            batch_count+=1
        # 测试集上的正确率
        test_acc=evaluate_accuracy(test_iter,net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
        %(epoch+1,train_l_sum/batch_count,train_acc_sum/n,test_acc,time.time()-start))    

In [19]:
# 学习率，迭代次数
lr,num_epochs=0.001,5
# 优化器，更新参数
optimizer=torch.optim.Adam(net.parameters(),lr=lr)

In [20]:
# train
train(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs)

training on  cpu
epoch 1, loss 1.8744, train acc 0.309, test acc 0.573, time 19.7 sec
epoch 2, loss 0.9330, train acc 0.649, test acc 0.697, time 18.9 sec
epoch 3, loss 0.7458, train acc 0.724, test acc 0.729, time 19.1 sec
epoch 4, loss 0.6667, train acc 0.743, test acc 0.746, time 19.7 sec
epoch 5, loss 0.6143, train acc 0.760, test acc 0.752, time 20.6 sec
